## Set Participant Number HERE

In [103]:
import pandas as pd

participant_number = 7903



## Define Functions

In [104]:
def get_sheet(sheet_name, sheet_type):
    try:
        df = pd.read_excel(f"{participant_number}.xlsx", sheet_name=sheet_name, header=None)
        if sheet_type == "mini":
            df.columns = ["WORD", "IS_CORRECT"]
        elif sheet_type == "training":
            df.columns = ["WORD", "IS_CORRECT", "RESPONSE_TIME"]
        elif sheet_type == "evaluation":
            df.columns = ["WORD", "IS_CORRECT", "NOISE_TYPE", "SPEAKER"]
        print(f"{sheet_name} sheet processed successfully.")
        return df
    except Exception as e:
        return None

def get_accuracy_average(df):    # cast to bool
    if df is None:
        return None
    else:
        percentage = df["IS_CORRECT"].value_counts().get(True, 0) / len(df) * 100
        # round to 2 decimal places
        return round(percentage, 2)

def get_accuracy_of_noise_type(df, noise_type):
    if df is None:
        return None
    else:
        noise_df = df[df["NOISE_TYPE"] == noise_type]
        return get_accuracy_average(noise_df)

## Load Sheets

In [105]:
sheet_type = 'evaluation'

df_pre = get_sheet("PRE", sheet_type)
df_mid = get_sheet("MID", sheet_type)
df_post = get_sheet("POST", sheet_type)

PRE sheet processed successfully.
MID sheet processed successfully.
POST sheet processed successfully.


In [106]:
sheet_type = 'training'

df_session1 = get_sheet("Session 1", sheet_type)
df_session2 = get_sheet("Session 2", sheet_type)
df_session3 = get_sheet("Session 3", sheet_type)
df_session4 = get_sheet("Session 4", sheet_type)
df_session5 = get_sheet("Session 5", sheet_type)
df_session6 = get_sheet("Session 6", sheet_type)
df_session7 = get_sheet("Session 7", sheet_type)
df_session8 = get_sheet("Session 8", sheet_type)

Session 1 sheet processed successfully.
Session 2 sheet processed successfully.
Session 3 sheet processed successfully.
Session 4 sheet processed successfully.
Session 5 sheet processed successfully.
Session 6 sheet processed successfully.
Session 7 sheet processed successfully.
Session 8 sheet processed successfully.


In [107]:
sheet_type = 'mini'

df_mini1 = get_sheet("Mini 1", sheet_type)
df_mini2 = get_sheet("Mini 2", sheet_type)
df_mini3 = get_sheet("Mini 3", sheet_type)
df_mini4 = get_sheet("Mini 4", sheet_type)
df_mini5 = get_sheet("Mini 5", sheet_type)
df_mini6 = get_sheet("Mini 6", sheet_type)
df_mini7 = get_sheet("Mini 7", sheet_type)
df_mini8 = get_sheet("Mini 8", sheet_type)

Mini 1 sheet processed successfully.
Mini 2 sheet processed successfully.
Mini 3 sheet processed successfully.
Mini 4 sheet processed successfully.
Mini 5 sheet processed successfully.
Mini 6 sheet processed successfully.
Mini 7 sheet processed successfully.
Mini 8 sheet processed successfully.


## Compare PRE, MID, POST

In [108]:
pre_acc = get_accuracy_average(df_pre)
mid_acc = get_accuracy_average(df_mid)
post_acc = get_accuracy_average(df_post)

result = {
    "PRE": pre_acc,
    "MID": mid_acc,
    "POST": post_acc
}

print(f'Average Accuracy: {result}')

Average Accuracy: {'PRE': np.float64(75.69), 'MID': np.float64(82.64), 'POST': np.float64(76.39)}


### Compare Noise Types within PRE/MID/POST

In [109]:
noise_types = ["Clear", "PinkNoise", "SingleTalker"]

pre_acc = { noise_type: get_accuracy_of_noise_type(df_pre, noise_type) for noise_type in noise_types }
mid_acc = { noise_type: get_accuracy_of_noise_type(df_mid, noise_type) for noise_type in noise_types }
post_acc = { noise_type: get_accuracy_of_noise_type(df_post, noise_type) for noise_type in noise_types }

In [110]:
print(f'PRE: {pre_acc}')
print(f'MID: {mid_acc}')
print(f'POST: {post_acc}')

PRE: {'Clear': np.float64(81.25), 'PinkNoise': np.float64(66.67), 'SingleTalker': np.float64(79.17)}
MID: {'Clear': np.float64(83.33), 'PinkNoise': np.float64(83.33), 'SingleTalker': np.float64(81.25)}
POST: {'Clear': np.float64(79.17), 'PinkNoise': np.float64(70.83), 'SingleTalker': np.float64(79.17)}


## Compare Sessions

In [111]:
def get_valid_sessions():
    sessions = [df_session1, df_session2, df_session3, df_session4, df_session5, df_session6, df_session7, df_session8]
    return [df for df in sessions if df is not None]

In [112]:
sessions = get_valid_sessions()

session_accuracies = []
for session in sessions:
    session_accuracies.append(get_accuracy_average(session))

result = { f"Session {i+1}": session_accuracies[i] for i in range(len(session_accuracies))}

print(result)

{'Session 1': np.float64(67.86), 'Session 2': np.float64(72.32), 'Session 3': np.float64(66.96), 'Session 4': np.float64(77.68), 'Session 5': np.float64(72.32), 'Session 6': np.float64(78.57), 'Session 7': np.float64(80.36), 'Session 8': np.float64(79.46)}


## Compare Minis

In [113]:
def get_valid_minis():
    minis = [df_mini1, df_mini2, df_mini3, df_mini4, df_mini5, df_mini6, df_mini7, df_mini8]
    return [df for df in minis if df is not None]

In [114]:
minis = get_valid_minis()

mini_accuracies = []
for mini in minis:
    mini_accuracies.append(get_accuracy_average(mini))

result = { f"Mini {i+1}": mini_accuracies[i] for i in range(len(mini_accuracies))}

print(result)

{'Mini 1': np.float64(100.0), 'Mini 2': np.float64(62.5), 'Mini 3': np.float64(87.5), 'Mini 4': np.float64(100.0), 'Mini 5': np.float64(100.0), 'Mini 6': np.float64(87.5), 'Mini 7': np.float64(62.5), 'Mini 8': np.float64(100.0)}


# Detailed Analysis - Word Level

### Calculate Word Accuracy(0, 0.5, 1) Across Sessions

In [115]:
def calculate_accuracy(df):
    accuracy_dict = {}
    grouped = df.groupby('WORD')
    for w, group in grouped:
        accuracy = group['IS_CORRECT'].mean()
        accuracy_dict[w] = accuracy
    acc_df = pd.DataFrame(list(accuracy_dict.items()), columns=['WORD', 'ACCURACY'])
    return acc_df


def track_accuracies_by_words(dfs, is_eval=False):
    acc_df = pd.DataFrame()
    if is_eval:
        session_names = ['PRE', 'MID', 'POST']
    else:
        session_names = [f"Session {i+1}" for i in range(len(dfs))]

    for df, session_name in zip(dfs, session_names):
        # leave only first 2 columns - word, is_correct
        session_accuracy = calculate_accuracy(df)
        session_accuracy.columns = ['WORD', session_name]
        if acc_df.empty:
            acc_df = session_accuracy
        else:
            acc_df = pd.merge(acc_df, session_accuracy, on='WORD', how='outer')

    if not acc_df.empty:
        acc_df = acc_df.sort_values(by='WORD', key=lambda x: x.str.lower())

    return acc_df

from openpyxl import load_workbook

def save_accuracy_to_excel(df, sheet_name):
    excel_file = f"{participant_number}.xlsx"

    try:
        with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"{sheet_name} sheet is saved to {excel_file}")
    except ValueError as e:
        if 'Sheet' in str(e) and 'already exists' in str(e):
            with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl') as writer:
                workbook = load_workbook(excel_file)
                if 'Word Accuracies' in workbook.sheetnames:
                    del workbook[sheet_name]
                    workbook.save(excel_file)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
    except Exception as e:
        print(f"An error occurred: {e}")

In [116]:
accuracy_df = track_accuracies_by_words(sessions)
save_accuracy_to_excel(accuracy_df, 'Word Accuracies')

accuracy_df_eval = track_accuracies_by_words([df_pre, df_mid, df_post], is_eval=True)
save_accuracy_to_excel(accuracy_df_eval, 'Evaluation Word Accuracies')

Word Accuracies sheet is saved to 7903.xlsx
Evaluation Word Accuracies sheet is saved to 7903.xlsx


### Calculate Vowel Contrast Accuracy Across Sessions

In [117]:
def get_contrast(word):
    contrast_dict = {
        **dict.fromkeys(["rich", "reach", "itch", "each", "sin", "scene", "list", "least", "chip", "cheap", "filled", "field", "grin", "green"], "ɪ vs. iː"),
        **dict.fromkeys(["bet", "bat", "pet", "pat", "met", "mat", "set", "sat", "ten", "tan", "men", "man", "Ken", "can"], "ɛ vs. æ"),
        **dict.fromkeys(["cut", "cot", "but", "bot", "hut", "hot", "nut", "not", "sub", "sob", "fund", "fond", "pup", "pop"], "ʌ vs. ɑ"),
        **dict.fromkeys(["look", "Luke", "pull", "pool", "full", "fool", "should", "shooed", "bull", "Boole", "could", "cooed", "would", "wooed"], "ʊ vs. uː")
    }

    return contrast_dict.get(word, "Word not found in the table")

In [118]:
# for each word, get the contrast, and across sessions, calculate the accuracy of each contrast. save to excel sheet.
def calculate_contrast_accuracy(df):
    contrast_dict = {}
    grouped = df.groupby('WORD')
    for w, group in grouped:
        contrast = get_contrast(w)
        accuracy = group['IS_CORRECT'].mean()
        if contrast in contrast_dict:
            contrast_dict[contrast].append(accuracy)
        else:
            contrast_dict[contrast] = [accuracy]

    contrast_accuracy_dict = {}
    for contrast, accuracies in contrast_dict.items():
        contrast_accuracy_dict[contrast] = sum(accuracies) / len(accuracies)

    acc_df = pd.DataFrame(list(contrast_accuracy_dict.items()), columns=['CONTRAST', 'ACCURACY'])
    return acc_df


def track_contrast_accuracies_by_words(dfs):
    acc_df = pd.DataFrame()
    session_names = [f"Session {i+1}" for i in range(len(dfs))]

    for df, session_name in zip(dfs, session_names):
        # leave only first 2 columns - word, is_correct
        session_accuracy = calculate_contrast_accuracy(df)
        session_accuracy.columns = ['CONTRAST', session_name]
        if acc_df.empty:
            acc_df = session_accuracy
        else:
            acc_df = pd.merge(acc_df, session_accuracy, on='CONTRAST', how='outer')

    if not acc_df.empty:
        acc_df = acc_df.sort_values(by='CONTRAST', key=lambda x: x.str.lower())

    return acc_df


contrast_accuracy_df = track_contrast_accuracies_by_words(sessions)

save_accuracy_to_excel(contrast_accuracy_df, 'Contrast Accuracies')

Contrast Accuracies sheet is saved to 7903.xlsx
